<a href="https://colab.research.google.com/github/alejandrodgb/fastai/blob/main/clean/04_MNIST_full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys

if 'google.colab' in sys.modules:
    ! [ -e /content ] && pip install -Uqq fastbook
    !pip install nbdev
    
import fastbook
fastbook.setup_book()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 KB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 75.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 75.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 84.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 19.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 KB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [2]:
from fastai.vision.all import *
from fastbook import *

# Data

In [3]:
# Download full MNIST dataset
path = untar_data(URLs.MNIST)
path

Path('/root/.fastai/data/mnist_png')

# Fastai classifier

In [4]:
# Build data loader
dls = ImageDataLoaders.from_folder(path,train='training' ,valid='testing')

In [5]:
learn = vision_learner(dls,resnet18,pretrained=False,
                       loss_func=F.cross_entropy,metrics=accuracy)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [6]:
learn.fit_one_cycle(2,0.1)

epoch,train_loss,valid_loss,accuracy,time
0,0.694914,0.378404,0.939600,01:35
1,0.063946,1.096199,0.989600,01:24


# Custom trainer

## Data

In [7]:
# Review directories
training_path = (path/'training').ls().sorted()
valid_path = (path/'testing').ls().sorted()

In [8]:
# Check training data
av_imgs = [len(i.ls()) for i in training_path]
av_imgs, sum(av_imgs)

([5923, 6742, 5958, 6131, 5842, 5421, 5918, 6265, 5851, 5949], 60000)

In [9]:
# Check testing data
av_imgs = [len(i.ls()) for i in valid_path]
av_imgs, sum(av_imgs)

([980, 1135, 1032, 1010, 982, 892, 958, 1028, 974, 1009], 10000)

In [10]:
# List of lists of image paths
train_nums = [n.ls() for n in training_path]
valid_nums = [n.ls() for n in valid_path]

In [11]:
# Create a list of tensors for each digit
train_nums_lists = [[tensor(Image.open(im)) for im in num] for num in train_nums]
valid_nums_lists = [[tensor(Image.open(im)) for im in num] for num in valid_nums]

In [12]:
# Stack all tensors, convert to float, and normalise
train_nums_tensors = [torch.stack(nums_list).float()/255 for nums_list in train_nums_lists]
valid_nums_tensors = [torch.stack(nums_list).float()/255 for nums_list in valid_nums_lists]

In [13]:
# Check test tensors
av_t = [t.shape for t in train_nums_tensors]
av_t,[i[0] for i in av_t], sum([i[0] for i in av_t])

([torch.Size([5923, 28, 28]),
  torch.Size([6742, 28, 28]),
  torch.Size([5958, 28, 28]),
  torch.Size([6131, 28, 28]),
  torch.Size([5842, 28, 28]),
  torch.Size([5421, 28, 28]),
  torch.Size([5918, 28, 28]),
  torch.Size([6265, 28, 28]),
  torch.Size([5851, 28, 28]),
  torch.Size([5949, 28, 28])],
 [5923, 6742, 5958, 6131, 5842, 5421, 5918, 6265, 5851, 5949],
 60000)

In [14]:
# Check valid tensors
av_t = [t.shape for t in valid_nums_tensors]
av_t,[i[0] for i in av_t], sum([i[0] for i in av_t])

([torch.Size([980, 28, 28]),
  torch.Size([1135, 28, 28]),
  torch.Size([1032, 28, 28]),
  torch.Size([1010, 28, 28]),
  torch.Size([982, 28, 28]),
  torch.Size([892, 28, 28]),
  torch.Size([958, 28, 28]),
  torch.Size([1028, 28, 28]),
  torch.Size([974, 28, 28]),
  torch.Size([1009, 28, 28])],
 [980, 1135, 1032, 1010, 982, 892, 958, 1028, 974, 1009],
 10000)

In [15]:
# Transform rank-3 to rank-2 tensors
train_x = torch.cat(train_nums_tensors).view(-1,28*28)
valid_x = torch.cat(valid_nums_tensors).view(-1,28*28)

train_x.shape, valid_x.shape

(torch.Size([60000, 784]), torch.Size([10000, 784]))

In [16]:
# Create target labels
train_y = torch.cat([tensor([i]*s).unsqueeze(1) for i,s in enumerate([t.shape[0] for t in train_nums_tensors])])
valid_y = torch.cat([tensor([i]*s).unsqueeze(1) for i,s in enumerate([t.shape[0] for t in valid_nums_tensors])])

train_y.shape, valid_y.shape

(torch.Size([60000, 1]), torch.Size([10000, 1]))

In [17]:
# Create validation and testing PyTorch datasets
train_dset = list(zip(train_x, train_y))
valid_dset = list(zip(valid_x, valid_y))

In [18]:
# Create data loader
train_dl = DataLoader(train_dset, bs=64, shuffle=True)

## Simple net model

In [19]:
# Multiclass loss
def mnist_loss(preds, targs):
    preds = preds.softmax(dim=1)
    return -(torch.log(preds[np.arange(len(preds)),targs.squeeze()])).mean()


In [20]:
# Create optimiser
class BasicOptim:
    def __init__(self,parameters,lr):
        self.params = list(parameters)
        self.lr = lr
        
    def step(self, *args, **kwargs):
        for p in self.params:
            p.data -= self.lr * p.grad.data
            
    def zero_grad(self, *args, **kwargs):
        for p in self.params:
            p.grad = None

In [21]:
def batch_accuracy(preds, y):
    return (preds.argmax(dim=preds.dim()-1)==y.squeeze()).float().mean()

In [22]:
def calc_grads(model, x, y, loss_fn):
    preds = model(x)
    loss = loss_fn(preds,y.squeeze())
    loss.backward()
    return loss.item()

In [23]:
def validate_epoch(model, valid_dl):
    accs = [batch_accuracy(model(xb),yb) for xb,yb in valid_dl]
    return round(torch.stack(accs).mean().item(),4)

In [24]:
def train_epoch(dl, model, loss_fn, opt):
    batch_loss = []
    for xb, yb in dl:
        batch_loss.append(calc_grads(model, xb, yb, loss_fn))
        opt.step()
        opt.zero_grad()
    print(f'\tAvg batch loss: {tensor(batch_loss).mean()}')

In [25]:
# Train model
def train_model(dl, model, epochs, loss_fn, valid_dl, opt):
    model_start = time.time()
    for e in range(epochs):
        epoch_start = time.time()
        train_epoch(dl, model, loss_fn, opt)
        print(f'Epoch {e}: accuracy = {validate_epoch(model, valid_dl)}, time {time.time()-epoch_start:.2f}s')
    print(f'Training time: {time.time()-model_start:.2f}s')

### Testing

In [26]:
# Simple neural network
simple_net = nn.Sequential(
    nn.Linear(28*28,100),
    nn.ReLU(),
    nn.Linear(100,30),
    nn.ReLU(),
    nn.Linear(30,10)
)
opt = BasicOptim(simple_net.parameters(), 0.1)

In [27]:
for xb, yb in train_dl:
    break

#### Categorical $L_{CE}$

In [28]:
preds = simple_net(xb)
preds.shape

torch.Size([64, 10])

In [29]:
num_classes=len(preds) if preds.dim()==1 else preds.shape[1]
num_classes

10

In [30]:
targs = F.one_hot(yb,num_classes=num_classes).view(-1,num_classes)
targs.shape

torch.Size([64, 10])

In [31]:
s_preds = preds.softmax(dim=1)

In [32]:
targs.shape, s_preds.shape

(torch.Size([64, 10]), torch.Size([64, 10]))

In [33]:
-(targs*torch.log(s_preds)).sum(dim=1).mean()

tensor(2.3108, grad_fn=<NegBackward0>)

#### Sparce $L_{CE}$

In [34]:
s_preds = preds.softmax(dim=1)

In [35]:
-(torch.log(s_preds[np.arange(len(s_preds)),yb.squeeze()])).mean()

tensor(2.3108, grad_fn=<NegBackward0>)

#### PyTorch $L_{CE}$

In [36]:
tcel = nn.CrossEntropyLoss()
tcel(preds,yb.squeeze())

tensor(2.3108, grad_fn=<NllLossBackward0>)

### Model training

In [37]:
# Simple neural network
simple_net = nn.Sequential(
    nn.Linear(28*28,100),
    nn.ReLU(),
    nn.Linear(100,30),
    nn.ReLU(),
    nn.Linear(30,10)
)
opt = BasicOptim(simple_net.parameters(), 0.1)

#### PyTorch $L_{CE}$

In [38]:
train_model(dl=train_dl, model=simple_net, epochs=10, loss_fn=nn.CrossEntropyLoss(), 
            valid_dl=valid_dset, opt=opt)

	Avg batch loss: 0.5231608152389526
Epoch 0: accuracy = 0.9317, time 3.85s
	Avg batch loss: 0.18905791640281677
Epoch 1: accuracy = 0.9469, time 2.81s
	Avg batch loss: 0.13389848172664642
Epoch 2: accuracy = 0.9653, time 2.85s
	Avg batch loss: 0.10432102531194687
Epoch 3: accuracy = 0.9639, time 3.02s
	Avg batch loss: 0.0852615088224411
Epoch 4: accuracy = 0.9699, time 3.86s
	Avg batch loss: 0.07128070294857025
Epoch 5: accuracy = 0.9699, time 3.59s
	Avg batch loss: 0.06071796268224716
Epoch 6: accuracy = 0.9722, time 2.76s
	Avg batch loss: 0.05313842371106148
Epoch 7: accuracy = 0.9667, time 2.89s
	Avg batch loss: 0.045916974544525146
Epoch 8: accuracy = 0.9769, time 2.77s
	Avg batch loss: 0.03915255889296532
Epoch 9: accuracy = 0.9744, time 3.98s
Training time: 32.38s


#### mnist_loss $L_{CE}$

In [39]:
# Simple neural network
simple_net = nn.Sequential(
    nn.Linear(28*28,100),
    nn.ReLU(),
    nn.Linear(100,30),
    nn.ReLU(),
    nn.Linear(30,10)
)
opt = BasicOptim(simple_net.parameters(), 0.1)

In [40]:
train_model(dl=train_dl, model=simple_net, epochs=10, loss_fn=mnist_loss,
            valid_dl=valid_dset, opt=opt)

	Avg batch loss: 0.5196418762207031
Epoch 0: accuracy = 0.9263, time 3.62s
	Avg batch loss: 0.19890545308589935
Epoch 1: accuracy = 0.9508, time 2.92s
	Avg batch loss: 0.13962110877037048
Epoch 2: accuracy = 0.9574, time 3.03s
	Avg batch loss: 0.10747168213129044
Epoch 3: accuracy = 0.9629, time 3.05s
	Avg batch loss: 0.08718109130859375
Epoch 4: accuracy = 0.9587, time 4.28s
	Avg batch loss: 0.07299192249774933
Epoch 5: accuracy = 0.9711, time 3.08s
	Avg batch loss: 0.060539163649082184
Epoch 6: accuracy = 0.9729, time 2.83s
	Avg batch loss: 0.05304195359349251
Epoch 7: accuracy = 0.9729, time 3.05s
	Avg batch loss: 0.04544098302721977
Epoch 8: accuracy = 0.9763, time 3.47s
	Avg batch loss: 0.03918003663420677
Epoch 9: accuracy = 0.973, time 4.26s
Training time: 33.60s
